# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE

[nltk_data] Downloading package punkt to /home/brunom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/brunom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/brunom/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [3]:
# load data from database
engine = create_engine('sqlite:///data/disasterResponse.db')
df = pd.read_sql('SELECT * FROM messages_categories', engine)

In [4]:
df.head()

,id,message,original,genre,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,fire,floods,food,hospitals,infrastructure_related,medical_help,medical_products,military,missing_people,money,offer,other_aid,other_infrastructure,other_weather,refugees,related,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [5]:
# preliminary cleaning
df.iloc[:,4:] = df.iloc[:,4:].astype('int64')
df.drop('related', axis=1, inplace=True)

### 2. Write a tokenization function to process your text data

In [6]:
stop_words = set(stopwords.words('english'))
def tokenize(sentence):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    return [lemmatizer.lemmatize(word).lower().strip() for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

message = df.message.apply(lambda x: " ".join(tokenize(x)))

In [7]:
message[:10][0]

'weather update cold front cuba could pas haiti'

#### Classes are very imbalanced

In [9]:
print(df.shape)
df.iloc[:,4:].apply(lambda x: x.value_counts())

(26216, 39)


,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,fire,floods,food,hospitals,infrastructure_related,medical_help,medical_products,military,missing_people,money,offer,other_aid,other_infrastructure,other_weather,refugees,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,25907,15356,24883,26216.0,25811,25686,25022,21141,23761,25684,25934,24061,23293,25933,24511,24132,24903,25356,25918,25612,26098,22770,25065,24840,25341,21742,25492,25745,23902,26096,23773,26057,25015,24544,18919
1,309,10860,1333,NaN,405,530,1194,5075,2455,532,282,2155,2923,283,1705,2084,1313,860,298,604,118,3446,1151,1376,875,4474,724,471,2314,120,2443,159,1201,1672,7297


In [10]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
aid_centers                   0
aid_related                   0
buildings                     0
child_alone                   0
clothing                      0
cold                          0
death                         0
direct_report                 0
earthquake                    0
electricity                   0
fire                          0
floods                        0
food                          0
hospitals                     0
infrastructure_related        0
medical_help                  0
medical_products              0
military                      0
missing_people                0
money                         0
offer                         0
other_aid                     0
other_infrastructure          0
other_weather                 0
refugees                      0
request                       0
search_and_rescue             0
security

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
class Clean_text(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X=None, y=None):
        return self
    
    def transform(self, X, y=None):
        
        return pd.Series(X).apply(lambda x: " ".join(self.tokenize(x))).values
    
    def tokenize(self, sentence):
        #stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        return [lemmatizer.lemmatize(word).lower().strip() for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [12]:
clean_text = Clean_text()
clean_text.transform(df.message)[:5]

array(['weather update cold front cuba could pas haiti', 'is hurricane',
       'looking someone name',
       'un report leogane destroyed only hospital st croix functioning needs supply desperately',
       'say west side haiti rest country today tonight'], dtype=object)

In [13]:
ml_pipeline = Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('abclf', MultiOutputClassifier(AdaBoostClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X = df.message.copy()
y = df.iloc[:,4:].values 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32) # training
X_train.shape, y_train.shape
ml_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('clean_text', Clean_text()), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

#### Because classes are imbalanced, accuracy is not a good metric for this problem.

In [16]:
X_train.head()

23313    Forty-eight teams of medical and paramedical s...
7683                   NOTES: this message is not complete
23462    All graphs provide evidence that, considering ...
22334    construction material: bricks - 500 t; cement ...
6522     I can arrange shelter shelter, i live out in t...
Name: message, dtype: object

In [17]:
def cross_val(X_train, y_train, pipeline):
    labels = []
    preds = []
    kf = KFold(n_splits=5, shuffle=True)
    for train_indices, test_indices in kf.split(X_train, y_train):
   
        X_train_kf = X_train.iloc[train_indices]
        y_train_kf = y_train[train_indices]
    
        X_test_kf = X_train.iloc[test_indices]
        y_test_kf = y_train[test_indices]
  
        pipeline.fit(X_train_kf, y_train_kf)
    
                
        labels.extend(y_test_kf)
        preds.extend(pipeline.predict(X_test_kf))

    return pd.DataFrame(labels), pd.DataFrame(preds)
    

In [27]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

labels_df, preds_df = cross_val(X_res, y_res, ml_pipeline)

ValueError: could not convert string to float: 'would like to make food/ serve food'

In [19]:
def scoring(scoring_metric, labels_df, preds_df):
    return np.mean(np.array([scoring_metric(labels_df.iloc[:,column], preds_df.iloc[:,column]) 
            for column in range(0,len(df.iloc[:,4:].columns))]))

list_scores = [f1_score, accuracy_score, precision_score, recall_score]
{str(metric.__name__):scoring(metric, labels_df, preds_df) for metric in list_scores}

{'f1_score': 0.38331031369442364,
 'accuracy_score': 0.9518296504182446,
 'precision_score': 0.5477374941118585,
 'recall_score': 0.30803428577420877}

In [20]:
y_preds = ml_pipeline.predict(X_test)

labels_df = pd.DataFrame(y_test)
preds_df = pd.DataFrame(y_preds)
list_scores = [f1_score, accuracy_score, precision_score, recall_score]
{str(metric.__name__):scoring(metric, labels_df, preds_df) for metric in list_scores}

{'f1_score': 0.369716694196754,
 'accuracy_score': 0.9509698158439578,
 'precision_score': 0.5393897866050884,
 'recall_score': 0.29508791625414005}

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
ml_pipeline.get_params()

{'memory': None,
 'steps': [('clean_text', Clean_text()),
  ('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('abclf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=None))],
 'clean_text': Clean_text(),
 'vectorizer': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max

In [ ]:
#parameters = {
#    'vectorizer__max_df': [1.0,2.0],
#    'vectorizer__max_features': [None,5,10],
#    'vectorizer__ngram_range': [(1, 1),(1,2)],
#    'tfidf__norm': ['l2','l1'],
#    'tfidf__smooth_idf': [True, False],
#    'tfidf__sublinear_tf': [True, False],
#    'abclf__estimator__learning_rate': [0.1,0.2,0.5],
#    'abclf__estimator__n_estimators': [50,100]
#}

parameters = {
    #'vectorizer__min_df': [1],
    #'vectorizer__max_df': [1.0],
    #'vectorizer__max_features': [None],
    #'vectorizer__ngram_range': [(1,1)],
    #'tfidf__norm': ['l2'],
    #'tfidf__smooth_idf': [True],
    #'tfidf__sublinear_tf': [False],
    'abclf__estimator__learning_rate': [0.1,0.2],
    'abclf__estimator__n_estimators': [100]
}


grid_clf = GridSearchCV(ml_pipeline, param_grid=parameters, verbose=10)

grid_clf.fit(X_train, y_train)

/home/brunom/.miniconda2/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100 
[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100, score=0.43827778572450293, total= 1.8min
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.0min remaining:    0.0s


[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=100, score=0.4380543633762518, total= 1.9min
[CV] abclf__estimator__learning_rate=0.2, abclf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  6.1min remaining:    0.0s


[CV]  abclf__estimator__learning_rate=0.2, abclf__estimator__n_estimators=100, score=0.4357030467744243, total= 1.8min
[CV] abclf__estimator__learning_rate=0.2, abclf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  8.1min remaining:    0.0s


[CV]  abclf__estimator__learning_rate=0.2, abclf__estimator__n_estimators=100, score=0.43598912887998853, total= 1.8min
[CV] abclf__estimator__learning_rate=0.2, abclf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 10.1min remaining:    0.0s


In [ ]:
ml_pipeline_best = grid_clf.best_estimator_
print(ml_pipeline_best)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model. Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
ml_pipeline_best = ml_pipeline
ml_pipeline_best.fit(X_train, y_train)
y_preds = ml_pipeline_best.predict(X_test)

labels_df = pd.DataFrame(y_test)
preds_df = pd.DataFrame(y_preds)
list_scores = [f1_score, accuracy_score, precision_score, recall_score]
{str(metric.__name__):scoring(metric, labels_df, preds_df) for metric in list_scores}

{'f1_score': 0.37569392022806103,
 'accuracy_score': 0.9520213577421816,
 'precision_score': 0.5482943938694825,
 'recall_score': 0.3020466938134612}

In [48]:
preds_df.apply(lambda x: x.value_counts())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,5244.0,3637,5244.0,5244.0,5244.0,5244.0,5244.0,4802,5244.0,5244.0,5244.0,5244.0,4745,5244.0,5244.0,5244.0,5244.0,5244.0,5244.0,5244.0,5244.0,5244.0,5244.0,5244.0,5244.0,4728,5244.0,5244.0,5239,5244.0,5244.0,5244.0,5244.0,5041,5244.0
1,NaN,1607,NaN,NaN,NaN,NaN,NaN,442,NaN,NaN,NaN,NaN,499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,516,NaN,NaN,5,NaN,NaN,NaN,NaN,203,NaN


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
class Clean_text_with_stemmer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X=None, y=None):
        return self
    
    def transform(self, X, y=None):
        
        return pd.Series(X).apply(lambda x: " ".join(self.tokenize(x))).values
    
    def tokenize(self, sentence):
        stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        return [stemmer.stem(lemmatizer.lemmatize(word).lower().strip()) for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [31]:
ml_pipeline_improved = Pipeline([
    ('clean_text', Clean_text_with_stemmer()),
    ('vectorizer', CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True, norm='l2')),
    ('mbclf', MultiOutputClassifier(LogisticRegression(random_state=34)))])

### 9. Export your model as a pickle file

In [ ]:
filename='ml_pipeline_improved.pkl'
pickle.dump(ml_pipeline_improved, open(filename,'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

## Scraps

In [250]:
# test y straight from df
vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
final_features = pd.DataFrame(vectorizer.fit_transform(df_clean_message).toarray(), 
                              columns=vectorizer.get_feature_names())

X = vector_features
y2 = MultiLabelBinarizer().fit_transform(labels)
y = df.iloc[:,4:].values
print(type(y2[0][0]))
print(type(y[0][0]))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)
print(y_train.shape)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y2, test_size=0.2, random_state=32)
print(y_train2.shape)
#clf = OneVsRestClassifier(SVC(probability=True))
#clf = OneVsRestClassifier(BernoulliNB())
clf = MultiOutputClassifier(BernoulliNB())
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

<class 'numpy.int64'>
<class 'numpy.int64'>
(20972, 36)
(20972, 35)


/home/brunom/.miniconda2/lib/python3.6/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


[0.9837909992372235,
 0.7688787185354691,
 0.918001525553013,
 1.0,
 0.9780701754385965,
 0.9702517162471396,
 0.9328756674294432,
 0.814836003051106,
 0.933257055682685,
 0.9687261632341724,
 0.9858886346300534,
 0.8983600305110603,
 0.9109458428680397,
 0.9832189168573608,
 0.8785278413424866,
 0.881578947368421,
 0.9248665141113653,
 0.9473684210526315,
 0.9837909992372235,
 0.9586193745232647,
 0.9921815408085431,
 0.8236079328756675,
 0.914187643020595,
 0.90255530129672,
 0.9363081617086194,
 0.7858504958047292,
 0.8773836765827613,
 0.9496567505720824,
 0.9725400457665904,
 0.906559877955759,
 0.9937070938215103,
 0.8991228070175439,
 0.9916094584286804,
 0.9204805491990846,
 0.9294431731502669]

#### Option 1 - use Tfidf

In [208]:
vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
final_features = pd.DataFrame(vectorizer.fit_transform(df_clean_message).toarray(), 
                              columns=vectorizer.get_feature_names())

display(final_features.head())

X = vector_features
y = MultiLabelBinarizer().fit_transform(labels)
print(len(labels))
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

#clf = OneVsRestClassifier(SVC(probability=True))
clf = OneVsRestClassifier(BernoulliNB())
#clf = OneVsRestClassifier(AdaBoostClassifier())
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

,aa,aai,ab,ababa,abad,abandon,abba,abdomin,abdomin pain,abduct,...,zhejiang provinc,zhouqu,zimbabw,zimbabwean,zinc,zinc roof,zone,zoonot,zoonot diseas,zy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


26216
(26216, 35)


[0.9855524734165511,
 0.7610957004160888,
 0.926490984743412,
 0.9808136846971799,
 0.9737632917244567,
 0.9379334257975035,
 0.8206195099398983,
 0.9248728617660656,
 0.9730698104484512,
 0.9880952380952381,
 0.9010633379565418,
 0.9054553860379103,
 0.9849745723532132,
 0.8839574664817383,
 0.8861534905224225,
 0.9290337494220989,
 0.9545769764216366,
 0.9845122515025427,
 0.9637078132223763,
 0.9931807674526121,
 0.8305594082293112,
 0.9218677762367082,
 0.9082293111419325,
 0.9467175219602404,
 0.7894128525196487,
 0.8769070735090153,
 0.9549237170596394,
 0.973994452149792,
 0.9060332871012483,
 0.9937586685159501,
 0.9008321775312067,
 0.9921405455386038,
 0.9239482200647249,
 0.929842810910772,
 0.8413083680073972]

#### Option 2: use CountVectorizer

In [203]:
vectorizer = CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
final_features = pd.DataFrame(vectorizer.fit_transform(df_clean_message).toarray(), 
                              columns=vectorizer.get_feature_names())

display(final_features.head())

X = vector_features
y = MultiLabelBinarizer().fit_transform(labels)
print(len(labels))
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

#clf = OneVsRestClassifier(SVC(probability=True))
clf = OneVsRestClassifier(BernoulliNB())
#clf = OneVsRestClassifier(AdaBoostClassifier())
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

,aa,aai,ab,ababa,abad,abandon,abba,abdomin,abdomin pain,abduct,...,zhejiang provinc,zhouqu,zimbabw,zimbabwean,zinc,zinc roof,zone,zoonot,zoonot diseas,zy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


26216
(26216, 35)


[0.9855524734165511,
 0.7610957004160888,
 0.926490984743412,
 0.9808136846971799,
 0.9737632917244567,
 0.9379334257975035,
 0.8206195099398983,
 0.9248728617660656,
 0.9730698104484512,
 0.9880952380952381,
 0.9010633379565418,
 0.9054553860379103,
 0.9849745723532132,
 0.8839574664817383,
 0.8861534905224225,
 0.9290337494220989,
 0.9545769764216366,
 0.9845122515025427,
 0.9637078132223763,
 0.9931807674526121,
 0.8305594082293112,
 0.9218677762367082,
 0.9082293111419325,
 0.9467175219602404,
 0.7894128525196487,
 0.8769070735090153,
 0.9549237170596394,
 0.973994452149792,
 0.9060332871012483,
 0.9937586685159501,
 0.9008321775312067,
 0.9921405455386038,
 0.9239482200647249,
 0.929842810910772,
 0.8413083680073972]

In [189]:
y.shape

(26216, 35)

In [128]:
#X = vector_features
y2 = df.iloc[:,4:-1].values
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.33, random_state=32)

In [163]:
def tokenize(sentence):
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    return [word for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 
df_clean_message = df.message.apply(lambda x: " ".join(tokenize(x)))
#vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l1', ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
X = vector_features
y = MultiLabelBinarizer().fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

#clf = OneVsRestClassifier(SVC(probability=True))
#clf = OneVsRestClassifier(BernoulliNB())
clf = OneVsRestClassifier(AdaBoostClassifier())
clf.fit(X_train, y_train)
clf.fit(X_train, y_trainb

accuracy_score(y_test, predictions)

0.2311604253351826

In [173]:
df.iloc[:,4:].sum(axis=0)

aid_centers                 309
aid_related               10860
buildings                  1333
child_alone                   0
clothing                    405
cold                        530
death                      1194
direct_report              5075
earthquake                 2455
electricity                 532
fire                        282
floods                     2155
food                       2923
hospitals                   283
infrastructure_related     1705
medical_help               2084
medical_products           1313
military                    860
missing_people              298
money                       604
offer                       118
other_aid                  3446
other_infrastructure       1151
other_weather              1376
refugees                    875
related                   20282
request                    4474
search_and_rescue           724
security                    471
shelter                    2314
shops                       120
storm   

In [175]:
len(df.iloc[:,4:].columns)

36

In [182]:
accuracy_score(y_test[:,0], predictions[:,0])

0.9878640776699029

In [184]:
y_test.shape

(8652, 35)

In [190]:
len(df.iloc[:,4:].columns)
[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

[0.9878640776699029,
 0.7457235321312992,
 0.9617429496070273,
 0.9872861766065649,
 0.9843966712898752,
 0.9664817383263985,
 0.8542533518261674,
 0.9696024040684235,
 0.9812760055478502,
 0.9912159038372631,
 0.9555016181229773,
 0.9435968562182154,
 0.9864771151178918,
 0.9294960702727693,
 0.9254507628294036,
 0.9526121128062875,
 0.9699491447064262,
 0.9878640776699029,
 0.9769995376791494,
 0.9939898289412853,
 0.8646555709662506,
 0.9520342117429496,
 0.9484512251502543,
 0.9684466019417476,
 0.7786638927415627,
 0.8919325011558021,
 0.970873786407767,
 0.9823162274618585,
 0.9469486823855756,
 0.994914470642626,
 0.9348127600554785,
 0.9942209893666204,
 0.9586222838650024,
 0.9605871474803513,
 0.8788719371243643]

In [175]:
clf = OneVsRestClassifier(MultinomialNB())
clf.fit(X_train, y_train)
preds = clf.predict(X_train) 

ValueError: Multioutput target data is not supported with label binarization

In [ ]:
OneVsRestClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X) 
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = BernoulliNB(alpha=1.0, class_prior=None, fit_prior=True)
clf.fit(X_train, y_train)


ValueError: bad input shape (17564, 37)

## Code Bank

In [32]:
# beautiful solution
#df.message.apply(lambda x: " ".join([stemmer.stem(word) for word in re.sub("[^a-zA-Z]", " ", x).split() 
#          if word not in stop_words]) )

In [42]:
class Select_features(BaseEstimator, TransformerMixin):
    # constructor
    def __init__(self, column_names):
        self.column_names = column_names 
    
    # pipeline must implement fit
    def fit(self, X=None, y=None):
        return self
    
    # return new dataframe with selected features
    def transform(self, X=None, y=None):
        return X[self.column_names] 

In [ ]:
X = vector_features
#y = MultiLabelBinarizer().fit_transform(labels)
#print(len(labels))
#print(y.shape)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)
#
#vectorizer = CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))
#clf = OneVsRestClassifier(BernoulliNB())
#message_pipeline = Pipeline([('vectorizer', vectorizer),('clf',clf)])
#message_pipeline.fit()
#message_pipeline.named_steps['vectorizer'].transform([df_clean_message])